## Úvod do regrese - Lekce 1

#### Uvedení do kontextu

✅ Existuje mnoho typů regresních metod a výběr té správné závisí na tom, jakou odpověď hledáte. Pokud chcete předpovědět pravděpodobnou výšku osoby určitého věku, použijete `lineární regresi`, protože hledáte **číselnou hodnotu**. Pokud vás zajímá, zda by určitý typ kuchyně měl být považován za veganský nebo ne, hledáte **přiřazení kategorie**, a proto byste použili `logistickou regresi`. O logistické regresi se dozvíte více později. Zamyslete se nad některými otázkami, které byste mohli klást datům, a nad tím, která z těchto metod by byla vhodnější.

V této části budete pracovat s [malou datovou sadou o cukrovce](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Představte si, že byste chtěli otestovat léčbu pro pacienty s cukrovkou. Modely strojového učení by vám mohly pomoci určit, kteří pacienti by na léčbu reagovali lépe, na základě kombinací proměnných. I velmi základní regresní model, pokud je vizualizován, by mohl ukázat informace o proměnných, které by vám pomohly uspořádat vaše teoretické klinické studie.

Tak tedy, pojďme se pustit do tohoto úkolu!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Ilustrace od @allison_horst</figcaption>

<!--![Ilustrace od \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Ilustrace od @allison_horst-->


## 1. Načtení našeho nástrojového setu

Pro tento úkol budeme potřebovat následující balíčky:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [kolekce balíčků pro R](https://www.tidyverse.org/packages), která je navržena tak, aby byla práce s daty rychlejší, jednodušší a zábavnější!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) je [kolekce balíčků](https://www.tidymodels.org/packages/) určená pro modelování a strojové učení.

Můžete je nainstalovat pomocí:

`install.packages(c("tidyverse", "tidymodels"))`

Níže uvedený skript zkontroluje, zda máte balíčky potřebné k dokončení tohoto modulu, a v případě, že některé chybí, je pro vás nainstaluje.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Nyní načtěme tyto skvělé balíčky a zpřístupněme je v naší aktuální relaci R. (Toto je pouze pro ilustraci, `pacman::p_load()` to již udělal za vás)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Datová sada o cukrovce

V tomto cvičení si procvičíme naše dovednosti v regresi tím, že budeme provádět predikce na datové sadě o cukrovce. [Datová sada o cukrovce](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) obsahuje `442 vzorků` dat týkajících se cukrovky, s 10 prediktivními proměnnými, jako jsou `věk`, `pohlaví`, `index tělesné hmotnosti`, `průměrný krevní tlak` a `šest měření krevního séra`, a také výstupní proměnnou `y`: kvantitativní míru progrese onemocnění jeden rok po výchozím stavu.

|Počet pozorování|442|
|----------------|:---|
|Počet prediktorů|Prvních 10 sloupců jsou číselné prediktivní proměnné|
|Výstup/Cíl|Sloupec 11 je kvantitativní míra progrese onemocnění jeden rok po výchozím stavu|
|Informace o prediktorech|- věk v letech
||- pohlaví
||- bmi index tělesné hmotnosti
||- bp průměrný krevní tlak
||- s1 tc, celkový cholesterol v séru
||- s2 ldl, lipoproteiny s nízkou hustotou
||- s3 hdl, lipoproteiny s vysokou hustotou
||- s4 tch, poměr celkového cholesterolu k HDL
||- s5 ltg, pravděpodobně logaritmus hladiny triglyceridů v séru
||- s6 glu, hladina cukru v krvi|




> 🎓 Pamatujte, že se jedná o učení s učitelem (supervised learning), a potřebujeme cílovou proměnnou 'y'.

Než budete moci manipulovat s daty v R, musíte data načíst do paměti R nebo vytvořit připojení k datům, které R umožní přístup k datům vzdáleně.

> Balíček [readr](https://readr.tidyverse.org/), který je součástí Tidyverse, poskytuje rychlý a uživatelsky přívětivý způsob, jak načíst obdélníková data do R.

Nyní načtěme datovou sadu o cukrovce z tohoto zdrojového URL: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Také provedeme kontrolu našich dat pomocí funkce `glimpse()` a zobrazíme prvních 5 řádků pomocí funkce `slice()`.

Než budeme pokračovat, představme si ještě něco, s čím se v kódu R často setkáte 🥁🥁: operátor pipe `%>%`

Operátor pipe (`%>%`) provádí operace v logickém sledu tím, že předává objekt dál do funkce nebo výrazu. Můžete si operátor pipe představit jako výraz "a pak" ve vašem kódu.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` nám ukazuje, že tato data obsahují 442 řádků a 11 sloupců, přičemž všechny sloupce mají datový typ `double`.

<br>

> glimpse() a slice() jsou funkce v [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, součást Tidyverse, je gramatika pro manipulaci s daty, která poskytuje konzistentní sadu sloves, jež vám pomohou vyřešit nejběžnější výzvy při práci s daty.

<br>

Teď, když máme data, zaměříme se na jednu vlastnost (`bmi`) jako cíl tohoto cvičení. To bude vyžadovat, abychom vybrali požadované sloupce. Jak to uděláme?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) nám umožňuje *vybrat* (a případně přejmenovat) sloupce v datovém rámci.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Trénovací a testovací data

V řízeném učení je běžnou praxí *rozdělit* data na dvě podmnožiny; (obvykle větší) sadu, která slouží k trénování modelu, a menší „záložní“ sadu, která slouží k ověření, jak si model vedl.

Nyní, když máme data připravena, můžeme zjistit, zda nám stroj dokáže pomoci určit logické rozdělení čísel v tomto datasetu. Můžeme použít balíček [rsample](https://tidymodels.github.io/rsample/), který je součástí frameworku Tidymodels, k vytvoření objektu, který obsahuje informace o *tom*, jak data rozdělit, a poté dvě další funkce rsample k extrakci vytvořených trénovacích a testovacích sad:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Trénování lineárního regresního modelu pomocí Tidymodels

Teď jsme připraveni trénovat náš model!

V Tidymodels specifikujete modely pomocí `parsnip()` tím, že určíte tři koncepty:

-   **Typ modelu** rozlišuje mezi modely, jako je lineární regrese, logistická regrese, modely rozhodovacích stromů a podobně.

-   **Režim modelu** zahrnuje běžné možnosti, jako je regrese a klasifikace; některé typy modelů podporují obě možnosti, zatímco jiné mají pouze jeden režim.

-   **Engine modelu** je výpočetní nástroj, který bude použit k nastavení modelu. Často se jedná o balíčky v R, jako například **`"lm"`** nebo **`"ranger"`**.

Tyto informace o modelování jsou zachyceny ve specifikaci modelu, tak si jednu vytvořme!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Poté, co byl model *specifikován*, může být model `odhadnut` nebo `natrénován` pomocí funkce [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), obvykle za použití vzorce a nějakých dat.

`y ~ .` znamená, že budeme přizpůsobovat `y` jako předpovídanou hodnotu/cíl, vysvětlenou všemi prediktory/vlastnostmi, tj. `.` (v tomto případě máme pouze jeden prediktor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Z výstupu modelu můžeme vidět koeficienty naučené během trénování. Tyto koeficienty představují parametry přímky nejlepšího přizpůsobení, která nám poskytuje nejnižší celkovou chybu mezi skutečnou a předpovězenou proměnnou.
<br>

## 5. Vytvoření předpovědí na testovací sadě

Nyní, když jsme model natrénovali, můžeme jej použít k předpovědi vývoje onemocnění y pro testovací dataset pomocí [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). To bude použito k vykreslení čáry mezi skupinami dat.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Hurá! 💃🕺 Právě jsme natrénovali model a použili ho k vytvoření předpovědí!

Při vytváření předpovědí je konvencí v tidymodels vždy vytvořit tibble/datový rámec s výsledky, který má standardizované názvy sloupců. To usnadňuje kombinaci původních dat a předpovědí do použitelného formátu pro následné operace, jako je například vytváření grafů.

`dplyr::bind_cols()` efektivně spojuje více datových rámců podle sloupců.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Zobrazení výsledků modelování

Teď je čas podívat se na to vizuálně 📈. Vytvoříme bodový graf všech hodnot `y` a `bmi` z testovací sady a poté použijeme predikce k nakreslení čáry na nejvhodnějším místě mezi skupinami dat modelu.

R má několik systémů pro vytváření grafů, ale `ggplot2` je jedním z nejpůsobivějších a nejvšestrannějších. Umožňuje vám skládat grafy **kombinováním nezávislých komponent**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Zamyslete se nad tím, co se zde děje. Přímka prochází mnoha malými body dat, ale co přesně dělá? Vidíte, jak byste měli být schopni použít tuto přímku k předpovědi, kam by měl nový, neviděný datový bod zapadnout ve vztahu k ose y grafu? Zkuste slovy popsat praktické využití tohoto modelu.

Gratulujeme, vytvořili jste svůj první model lineární regrese, provedli předpověď a zobrazili ji v grafu!



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Nenese odpovědnost za žádné nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
